In [4]:
# ============================================================
# Apriori algoritam – Amazon Fine Food Reviews
# Transakcije = skup proizvoda koje je isti korisnik ocenio
# Zadrzavaju se samo korisnici sa >= 2 proizvoda
# ============================================================

# !pip install -q mlxtend

import time
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# ------------------------------------------------------------
# Pomocna funkcija za logovanje vremena
# ------------------------------------------------------------
t0 = time.time()
def log(msg):
    print(f"[{time.time()-t0:6.1f}s] {msg}", flush=True)

# ------------------------------------------------------------
# 1) Ucitavanje podataka
# ------------------------------------------------------------
log("Ucitavam podatke...")
df = pd.read_csv("Reviews_sample_balanced.csv")
log(f"Ucitano redova: {len(df):,}")

# Provera potrebnih kolona
required_cols = {"UserId", "ProductId"}
if not required_cols.issubset(df.columns):
    raise ValueError(f"Nedostaju kolone: {required_cols - set(df.columns)}")

# ------------------------------------------------------------
# 2) Uklanjanje duplikata
# (isti korisnik – isti proizvod)
# ------------------------------------------------------------
log("Uklanjam duplikate (UserId, ProductId)...")
df = df.drop_duplicates(subset=["UserId", "ProductId"])
log(f"Nakon duplikata: {len(df):,} redova")

# ------------------------------------------------------------
# 3) Formiranje transakcija
# (lista ProductId po UserId)
# ------------------------------------------------------------
log("Formiram transakcije...")
transactions = (
    df.groupby("UserId")["ProductId"]
      .apply(lambda s: list(pd.unique(s)))
      .tolist()
)

log(f"Ukupno korisnika (transakcija): {len(transactions):,}")

# Analiza duzina transakcija
lens = pd.Series([len(t) for t in transactions])
print("\nStatistika duzine transakcija:")
print(lens.describe())
print("Korisnika sa 1 proizvodom:", (lens == 1).sum())
print("Korisnika sa >=2 proizvoda:", (lens >= 2).sum())

# ------------------------------------------------------------
# 4) Filtriranje korisnika sa >= 2 proizvoda
# ------------------------------------------------------------
log("Zadrzavam samo korisnike sa >=2 proizvoda...")
transactions2 = [t for t in transactions if len(t) >= 2]
log(f"Broj transakcija nakon filtriranja: {len(transactions2):,}")

# ------------------------------------------------------------
# 5) One-hot encoding (TransactionEncoder)
# ------------------------------------------------------------
log("Radim one-hot encoding...")
te = TransactionEncoder()
X = te.fit(transactions2).transform(transactions2)
basket = pd.DataFrame(X, columns=te.columns_)
log(f"One-hot dimenzije: {basket.shape[0]} x {basket.shape[1]}")

# ------------------------------------------------------------
# 6) Apriori – cesti skupovi stavki
# (fokus samo na parove proizvoda)
# ------------------------------------------------------------
min_support = 0.005   # ≈ 2 korisnika od 584
log(f"Pokrecem Apriori (min_support={min_support})...")

freq = apriori(
    basket,
    min_support=min_support,
    use_colnames=True,
    max_len=2,
    low_memory=True
)

freq["itemset_len"] = freq["itemsets"].apply(len)
log(f"Ukupno cistih itemset-ova: {len(freq):,}")
log(f"Parovi (len=2): {(freq['itemset_len'] == 2).sum():,}")

# ------------------------------------------------------------
# 7) Association rules
# ------------------------------------------------------------
log("Racunam asocijativna pravila...")
rules = association_rules(freq, metric="lift", min_threshold=1.1)
log(f"Ukupno pravila: {len(rules):,}")

# ------------------------------------------------------------
# 8) Priprema i prikaz rezultata
# ------------------------------------------------------------
def fs(s):
    return ", ".join(sorted(list(s)))

if len(rules) == 0:
    print("\nNEMA PRAVILA za izabrane pragove.")
else:
    rules = rules.sort_values(["lift", "confidence"], ascending=False)

    results = rules[
        ["antecedents", "consequents", "support", "confidence", "lift"]
    ].copy()

    results["antecedents"] = results["antecedents"].apply(fs)
    results["consequents"] = results["consequents"].apply(fs)

    print("\nTOP 10 ASOCIJATIVNIH PRAVILA:")
    print(results.head(20).to_string(index=False))


[   0.0s] Ucitavam podatke...
[   0.1s] Ucitano redova: 10,000
[   0.1s] Uklanjam duplikate (UserId, ProductId)...
[   0.2s] Nakon duplikata: 9,994 redova
[   0.2s] Formiram transakcije...
[   0.7s] Ukupno korisnika (transakcija): 9,252

Statistika duzine transakcija:
count    9252.000000
mean        1.080199
std         0.356312
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         8.000000
dtype: float64
Korisnika sa 1 proizvodom: 8668
Korisnika sa >=2 proizvoda: 584
[   0.7s] Zadrzavam samo korisnike sa >=2 proizvoda...
[   0.7s] Broj transakcija nakon filtriranja: 584
[   0.7s] Radim one-hot encoding...
[   0.7s] One-hot dimenzije: 584 x 1038
[   0.7s] Pokrecem Apriori (min_support=0.005)...
[   0.7s] Ukupno cistih itemset-ova: 71
[   0.7s] Parovi (len=2): 2
[   0.7s] Racunam asocijativna pravila...
[   0.8s] Ukupno pravila: 4

TOP 10 ASOCIJATIVNIH PRAVILA:
antecedents consequents  support  confidence       lift
 B001EQ596O  B001M050CU 0.00